In [ ]:
import cv2
import numpy as np
import cupy as cp
import torch
import time
from google.colab import files
from matplotlib import pyplot as plt
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap='gray')
plt.title("Original Image")
plt.axis("off")
plt.show()
def otsu_numpy(image):
    hist, _ = np.histogram(image.ravel(), 256, [0, 256])
    total = image.size
    sumB = 0.0
    wB = 0
    maximum = 0.0
    sum1 = np.dot(np.arange(256), hist)
       for t in range(256):
        wB += hist[t]
        if wB == 0:
            continue
        wF = total - wB
        if wF == 0:
            break
        sumB += t * hist[t]
        mB = sumB / wB
        mF = (sum1 - sumB) / wF
        between = wB * wF * (mB - mF) ** 2
        if between >= maximum:
            threshold = t
            maximum = between
    return threshold
def otsu_cupy(image):
    img_cp = cp.asarray(image, dtype=cp.uint8)
    hist_cp = cp.histogram(img_cp.ravel(), bins=256, range=(0, 256))[0]
    total = img_cp.size
    prob = hist_cp / total
    omega = cp.cumsum(prob)
    mu = cp.cumsum(cp.arange(256) * prob)
    mu_t = mu[-1]
    numerator = (mu_t * omega - mu) ** 2
    denominator = omega * (1.0 - omega)
    sigma_b_squared = cp.divide(numerator, denominator + 1e-10)
    threshold = int(cp.argmax(sigma_b_squared))
    return threshold
def otsu_pytorch_fast(image):
    img_tensor = torch.tensor(image, dtype=torch.float32).flatten()
    hist = torch.histc(img_tensor, bins=256, min=0, max=255)
    prob = hist / img_tensor.numel()
    levels = torch.arange(256, dtype=torch.float32)
    omega = torch.cumsum(prob, dim=0)
    mu = torch.cumsum(prob * levels, dim=0)
    mu_t = mu[-1]
    sigma_b_squared = ((mu_t * omega - mu) ** 2) / (omega * (1.0 - omega) + 1e-10)
    threshold = int(torch.argmax(sigma_b_squared))
    return threshold
def benchmark(method, name, repeat=3):
    start = time.time()
    for _ in range(repeat):
        threshold = method(img)
    duration = (time.time() - start) / repeat
    print(f"{name}: Threshold = {threshold}, Avg Time = {duration:.6f} s")
    return threshold, duration
print("Running Otsu's Threshold Benchmarks...\n")
t_np, time_np = benchmark(otsu_numpy, "NumPy Serial CPU")
t_cp, time_cp = benchmark(otsu_cupy, "CuPy GPU Parallel")
t_torch, time_torch = benchmark(otsu_pytorch_fast, "PyTorch CPU")
def print_speedup(cpu_time, other_time, label):
    speedup = cpu_time / other_time
    print(f"{label} Speedup: {speedup:.2f}x")
print("\n--- Speedups ---")
print_speedup(time_np, time_cp, "CuPy GPU")
print_speedup(time_np, time_torch, "PyTorch CPU
def apply_threshold(image, threshold):
    return (image > threshold).astype(np.uint8) * 255
img_np = apply_threshold(img, t_np)
img_cp = apply_threshold(img, t_cp)
img_torch = apply_threshold(img, t_torch)
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(img_np, cmap='gray')
plt.title(f"NumPy CPU (T={t_np})")
plt.axis("off")
plt.subplot(1, 3, 2)
plt.imshow(img_cp, cmap='gray')
plt.title(f"CuPy GPU (T={t_cp})")
plt.axis("off")
plt.subplot(1, 3, 3)
plt.imshow(img_torch, cmap='gray')
plt.title(f"PyTorch CPU (T={t_torch})")
plt.axis("off")
plt.tight_layout()
plt.show()